In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

Let's define some variables. 
<ul>
<li>The styles are the categories defined in the "1_image_download" notebook.</li>
<li> The image width and height is dicated by the type of network we will fine tune, in this case Resnet 50.</li>
<li>The batch size depends on your processor, it must be high enough to optimize speed and low enough not to run out of memory.</li>
<li>The number of epoch depends on how long it takes for the model to converge, in our case 100 epochs works</li>
<li>The count of categories, and of images in the train and validation dirs is dictated by the count of images previously downloade.</li>

In [ ]:
# define variables
styles = [
    'art deco', 'art nouveau', 'baroque',
    'louis xiii', 'neoclassical'
]
img_width, img_height = 256, 256
batch_size = 16
epochs = 100
train_dir = 'data/train'
validation_dir = 'data/validation'
n_cat = len(styles)
n_train = 150*5
n_val = 50*5

Load a pre-trained model, add custom layers, compile it.<br>
The callbacks are run afer each epoch. Checkpoint will save the best model, early will stop the training when the model converges.

In [ ]:
model = applications.ResNet50(weights="imagenet",
    include_top=False,
    input_shape = (img_width, img_height, 3)
)

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(n_cat, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), 
    metrics=["accuracy"]
)

# define callbacks
checkpoint = ModelCheckpoint("resnet_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model.summary()

Create generators to augment data and send it to the model from the train and validation directories.

In [ ]:
train_datagen = ImageDataGenerator(
rescale = 1./255, # makes pixel values be between [0,1] 
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Train models

In [ ]:
model_final.fit_generator(
train_generator,
samples_per_epoch = n_train,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = n_val,
callbacks = [checkpoint, early]
)

Test model

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from keras.applications import imagenet_utils

In [ ]:
test_model = load_model('resnet_1.h5')
dirs = [dirname for dirname in os.listdir('data/validation') if not dirname.startswith('.')]
styles = [
    'art deco', 'art nouveau', 'baroque',
    'louis xiii', 'neoclassical'
]

for dirname in dirs:
    counter = 1
    print('style to test')
    print(dirname)
    for file_name in os.listdir('data/validation/' + dirname):
        if not file_name.startswith('.'):
            img_path = os.path.join('data','validation',dirname,file_name)
            counter = counter + 1
            img_width, img_height = 256, 256
            img = load_img(img_path, target_size=(img_width, img_height))
            if img.mode != "RGB":
                img = img.convert("RGB")

            x = img_to_array(img)
            x = np.expand_dims(x, axis=0) # to make it nfeature, width, height, channels
            x = x/255
            prediction = test_model.predict(x)
            prediction = prediction.argmax(axis=-1)[0]
            print(styles[prediction])            
            if counter == 5:
                break


Save model architecture as json to save time when loading it

In [ ]:
model.save_weights('model.h5')
model_json = model.to_json()
with open('model.json', "w") as json_file:
    json_file.write(model_json)
json_file.close()